In [ ]:
这个比较器结合 LogSoftmax 和 NLLLoss实现。
当训练一个C类别分类问题时，它是非常有用的。如果提供了可选参数 weight（是一个一维的tensor），这将赋值给每个类别。在训练一个类别不均衡的数据集时，是非常有用的。


In [47]:
"""构造不同的input，比较其损失大小
01.查看标签平滑对计算损失得到的影响
"""
import torch as t
import torch.nn as nn
loss = nn.CrossEntropyLoss()
# 构造输入输出
input_1=t.tensor([[0.9238,0.1],[0.1, 0.912]])
input_2=t.tensor([[0.9999,0.001],[0.001, 0.9999]])

target = t.tensor([0, 1]) # 必须使用long类型

out_1 = loss(input_1, target)
out_2 = loss(input_2, target)
print(out_1) # 损失大
print(out_2) # 损失小


tensor(0.3656)
tensor(0.3136)


In [45]:
'''比较使用标签平滑与否得到的损失比较
02.
01.可以发现使用label_smooth 得到的损失较大

'''
bce = nn.BCELoss()
label = t.tensor([0.,1.])
label_smooth_1 = t.tensor([0.2,0.8]) # 使用一个标签平滑
label_smooth_2 = t.tensor([0.1,0.9])
pred = t.tensor([0.3,0.7])
loss_3 = bce(pred,label)
loss_4 = bce(pred,label_smooth_1)
loss_5 = bce(pred,label_smooth_2)

print(loss_3)
print(loss_4)
print(loss_5)

tensor(0.3567)
tensor(0.5261)
tensor(0.4414)


In [36]:
inp =t.tensor([[0.9999,0.001],[0.001, 0.9999]])
loss_fct = nn.BCELoss() # 计算交叉熵损失
softmax = nn.Softmax(dim=-1)
x = softmax(inp)
print(x)
xx_val,xx_idx = x.max(dim=-1)
print(xx_val)
y = t.tensor([0, 1]).float()

loss = loss_fct(xx_val,y)
print(loss)

tensor([[0.7308, 0.2692],
        [0.2692, 0.7308]])
tensor([0.7308, 0.7308])
tensor(0.8130)


In [19]:
import torch
import torch.nn as nn
# 如果在构造器中添加了 reduction = "none" 参数，则不再返回一个 scalar，也无法反向更新
loss = nn.CrossEntropyLoss()
input = torch.randn(3,5, requires_grad=True)

# 取3个数
# 需要注意，因为 input 的shape 是(3,5)，就表明只有5类，所以下面的类别只能是0-4，否则会报 “IndexError: Target 5 is out of bounds.” 错
# random_() 中的数的含义应该是指定一个到其范围内的数，这里就是生成大小为[0,5) 的整数
target = torch.empty(3, dtype=torch.long).random_(5)
print(input)
print(input.size())
print("-------------------------")
print(target)
print(target.size())
output = loss(input, target)
print(output.size())
output.backward() 
print(output)

tensor([[-0.5651,  0.2735,  0.2883,  0.7495,  0.4461],
        [-0.4567,  0.9265,  0.8855,  0.5502,  1.2194],
        [ 1.4972, -1.1494,  0.6773, -0.2750, -0.0401]], requires_grad=True)
torch.Size([3, 5])
-------------------------
tensor([1, 2, 1])
torch.Size([3])
torch.Size([])
tensor(2.1429, grad_fn=<NllLossBackward>)


In [32]:
def get_rid_of_number_in_str(string):
    deleted = [1] * len(string)
    res = "" 
    pre_char = '' # 上一个字符
    for i,char in enumerate(string):
        if char.isdigit()  and pre_char.isdigit():# 如果当前是数字，且之前也是数字
            deleted[i] = 0 # 记为待删除
        #print(char,end='')
        pre_char = char
    
    for i in range(len(string)):
        if deleted[i]:
            res+=string[i]
    return res
res = get_rid_of_number_in_str("3是类别数, 512 是计算损失的维度")
print(res)

3是类别数, 5 是计算损失的维度


In [26]:
a='1'
print(a.isdigit())

True


In [6]:
'''CrossEntropyLoss()函数的使用
'''
import torch as t
import torch.nn as nn

# 生成的size 是(4,3,5)。
# 对应到预测就是 4是batch，3是类别数, 5 是计算损失的维度。pred 代表预测的结果
pred = t.randn(4,3,5)  
gold = t.randint(3,(4,5)) # 范围是[0,3) ，也就是类别数；shape 是 (4,5) 。gold是标签
loss = nn.CrossEntropyLoss() # 计算损失
print(pred)
print(gold)
out1 = loss(pred,gold)

# 下面是使用第二种方法计算损失
# 思想是：先转换数据类型，然后再求解
print("----------------")
pred =pred.permute(0,2,1)  # 调整结构。在使用 view() 的时候是不会改变内存中数据的顺序的，但是permute()是会的
print(pred)
out2 = loss(pred.reshape(4*5,3),gold.view(5*4))
print(out1,out2)

tensor([[[ 0.2437, -0.3002, -1.8958, -0.8022, -1.1080],
         [-0.2849, -1.3770,  1.6620, -1.4426, -0.7600],
         [ 0.1275,  1.4556, -0.4783, -0.6048, -1.0159]],

        [[ 0.0476,  1.3372,  0.1179,  0.5213,  0.1213],
         [ 1.5183,  0.1490,  1.3848, -0.3382,  1.5580],
         [ 1.5705, -0.2842, -0.5677, -0.6010,  1.0007]],

        [[-0.0250, -0.9253,  0.0980, -0.0689, -0.3566],
         [-0.0494,  0.4148, -0.3458,  0.3093, -1.1003],
         [-1.2310,  0.7205,  0.9149,  1.2588, -0.2963]],

        [[ 0.0666, -0.4448, -0.3556, -0.6903,  0.1265],
         [ 0.9052,  1.0447,  0.4522, -0.7184,  0.2013],
         [-0.9177,  1.4244,  1.9818,  0.7684, -1.7831]]])
tensor([[0, 1, 0, 1, 0],
        [1, 2, 0, 2, 2],
        [2, 0, 2, 0, 1],
        [2, 2, 1, 0, 1]])
----------------
tensor([[[ 0.2437, -0.2849,  0.1275],
         [-0.3002, -1.3770,  1.4556],
         [-1.8958,  1.6620, -0.4783],
         [-0.8022, -1.4426, -0.6048],
         [-1.1080, -0.7600, -1.0159]],

        [[